In [19]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
import time
from features.models import ConvModelSmall
from features.settings import Baseline_Settings, Triplet_Hinge_Settings, Triplet_Log_Settings
from classes.models import LinearClassifier, TwoLayerClassifier
from learn import get_feature_model, get_classifier

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape((-1, 28, 28, 1))
x_test = x_test.reshape((-1, 28, 28, 1))

Using TensorFlow backend.


In [43]:
def get_save_path(i):
    return '/tmp/boostmodels' + str(i)

In [73]:
def train_classifiers(iterations):
    
    probs = np.ones(60000)/60000
    alpha = np.zeros(50)    
    
    for i in np.arange(iterations):
        
        tf.reset_default_graph()
        f, f_params = get_feature_model(feature_model=ConvModelSmall, settings=Triplet_Hinge_Settings,
                                        num_features=10, num_classes=10, data_format=[28, 28, 1],
                                        f_lr=0.07, f_reg=0.001, f_scope="f_cms_mnist_10")
        fs = tf.train.Saver(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, "f_cms_mnist_10"))
        (x, y, dropout, training) = f_params

        scope = "boosting" + str(i)
        c, features = get_classifier(classifier=TwoLayerClassifier, num_features=10, num_classes=10, 
                                    c_lr=0.02, c_reg=0.001, c_scope=scope, f_params=f_params)

        s = tf.train.Saver(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope))
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            fs.restore(sess, "./models/f_cms_mnist_10.ckpt")

            print("begin training classifier")
            train_time = time.time()
            for step in range(2001):
                indices = np.random.choice(60000, 1000, p=probs)
                x_ = x_train[indices]
                y_ = y_train[indices]
                features_ = sess.run(f.features, feed_dict={x:x_, y:y_, dropout:1.0, training:False})
                sess.run(c.optimize, feed_dict={x:x_, y:y_, features:features_, dropout:1.0, training:True})
                if step % 1000 == 0:
                    train_loss, train_acc = sess.run([c.loss, c.acc], feed_dict={x:x_, y:y_, features:features_, 
                                                                                 dropout:1.0, training:False})
                    print("\tepoch %d: train loss %g, train error %g"%(step/60, train_loss, 1 - train_acc))  
            train_time = time.time() - train_time
            print("end training classifier // time elapsed: %.4f s"%(train_time))

            eval_train_time = time.time()
            x_ = x_train
            y_ = y_train
            features_ = sess.run(f.features, feed_dict={x:x_, y:y_, dropout:1.0, training:False})
            incorrect = sess.run(c.incorrect, 
                                 feed_dict={x:x_, y:y_, features:features_, dropout:1.0, training:False})
            correct = incorrect * 2 - 1
            train_error = np.sum(probs[incorrect.astype(bool)])
            eval_train_time = time.time() - eval_train_time
            print("train set error: %.4f // time elapsed: %.4f s"%(train_error, eval_train_time))   

            if train_error < 0.5:
                s.save(sess=sess, save_path=get_save_path(i))
            else:
                return (i-1), alpha

            eval_test_time = time.time()
            features_ = sess.run(f.features, feed_dict={x:x_test, y:y_test, dropout:1.0, training:False})
            test_error = 1 - sess.run(c.acc, feed_dict={x:x_test, y:y_test, 
                                                        features:features_, dropout:1.0, training:False})
            eval_test_time = time.time() - eval_test_time
            print("test set error: %.4f // time elapsed: %.4f s"%(test_error, eval_test_time)) 

            alpha[i] = 0.5 * np.log((1 - train_error)/train_error)
            probs = probs * np.exp(alpha[i]*correct)
            probs /= np.sum(probs)
            
    return iterations, alpha

In [74]:
def eval_classifiers(iterations, alpha): 
    scores = np.zeros((10000, 10))
    
    for i in np.arange(iterations):
        
        tf.reset_default_graph()
        f, f_params = get_feature_model(feature_model=ConvModelSmall, settings=Triplet_Hinge_Settings,
                                        num_features=10, num_classes=10, data_format=[28, 28, 1],
                                        f_lr=0.07, f_reg=0.001, f_scope="f_cms_mnist_10")
        fs = tf.train.Saver(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, "f_cms_mnist_10"))
        (x, y, dropout, training) = f_params
    
        scope = "boosting" + str(i)
        c, features = get_classifier(classifier=TwoLayerClassifier, num_features=10, num_classes=10, 
                                    c_lr=0.02, c_reg=0.001, c_scope=scope, f_params=f_params)

        s = tf.train.Saver(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope))
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())

            s.restore(sess, get_save_path(i))
            features_ = sess.run(f.features, feed_dict={x:x_test, y:y_test, dropout:1.0, training:False})
            predictions = sess.run(c.predictions, feed_dict=
                                   {x:x_test, y:y_test, features:features_, dropout:1.0, training:False})
            scores[np.arange(10000), predictions] += alpha[i]
        
    return scores

In [75]:
iterations = 100
iterations, alpha = train_classifiers(iterations)
scores = eval_classifiers(iterations, alpha)
print("boosted accuracy:", np.mean(np.equal(np.argmax(scores, axis=1), y_test)))

INFO:tensorflow:Restoring parameters from ./models/f_cms_mnist_10.ckpt
begin training classifier
	epoch 0: train loss 2.61536, train error 0.882
	epoch 16: train loss 0.0753889, train error 0.011
	epoch 33: train loss 0.0873157, train error 0.015
end training classifier // time elapsed: 85.0288 s
train set error: 0.0126 // time elapsed: 0.9974 s
test set error: 0.0129 // time elapsed: 0.1920 s
INFO:tensorflow:Restoring parameters from ./models/f_cms_mnist_10.ckpt
begin training classifier
	epoch 0: train loss 2.39609, train error 0.885
	epoch 16: train loss 0.772468, train error 0.316
	epoch 33: train loss 0.690776, train error 0.293
end training classifier // time elapsed: 85.2076 s
train set error: 0.3012 // time elapsed: 0.9485 s
test set error: 0.0164 // time elapsed: 0.1747 s
INFO:tensorflow:Restoring parameters from ./models/f_cms_mnist_10.ckpt
begin training classifier
	epoch 0: train loss 2.39028, train error 0.854
	epoch 16: train loss 1.02831, train error 0.472
	epoch 33: tra